# ANOVA dumb group name - housing price prediction

<a href="#dummifying">Dummifying</a><br>

#### Contents
1. <a href="#target">Target</a><br>
2. <a href="#missingness">Missingness</a><br>
3. <a href="#cleaning">Data cleaning and preliminary feature engineering</a><br>
4. <a href="#modelling">Model fitting</a><br>
    a. <a href="#diagnostics">Diagnostics, multicollinearity, feature selection</a><br>
    b. <a href="#linear models">Linear models</a><br>
      - <a href="#optimal features">Optimal features and linear model re-fitting</a><br>
      - <a href="#linear model results">Linear model results</a><br>
 
 c. <a href="#non-linear models">Non-linear models</a><br>
 d. <a href="#all results">All Results</a><br>

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns
import statsmodels as sm

In [ ]:
# Read in the train and test data
train_data = pd.read_csv("data/train.csv", index_col = 0, encoding = "utf-8")
sale_price = train_data[['SalePrice']]
test_data = pd.read_csv("data/test.csv", index_col = 0, encoding = "utf-8")
test_data['SalePrice'] = 0

In [ ]:
# Combine the train and test data for the moment, so manipulations are applied to both
all_data = pd.concat([train_data, test_data], axis = 0).reset_index(drop = True)

<p><a name="target"></a></p>

###### Target is highly skewed! Use a transformation to correct

In [ ]:
# Compare normality of target under no transformation, optimum box-cox and log
fig1, ax1 = plt.subplots(figsize=(11.7, 8.27))
prob1 = stats.probplot(sale_price['SalePrice'], dist=stats.norm, plot=ax1)
ax1.set_title('Probplot against normal distribution')

print(all(sale_price['SalePrice'] > 0))
fig2, ax2 = plt.subplots(figsize=(11.7, 8.27))
sale_price_log = stats.boxcox(sale_price, lmbda = 0)
prob2 = stats.probplot(sale_price_log.flatten(), dist=stats.norm, plot=ax2)
ax2.set_title('Probplot after log transformation')

fig3, ax3 = plt.subplots(figsize=(11.7, 8.27))
sale_price_bc, lmbda = stats.boxcox(sale_price)
prob3 = stats.probplot(sale_price_bc.flatten(), dist=stats.norm, plot=ax3)
ax3.set_title('Probplot after Box-Cox transformation')

plt.show()
# Optimal Lambda is -0.07692396

In [ ]:
# Plot histograms of the lob and box-cos transformed target data
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(11.7, 8.27))
ax0, ax1, = axes.flatten()
ax0.hist(sale_price_log, density = True)
ax1.hist(sale_price_bc, density = True)

###### The target data transformed by the logarithm (lambda = 0) looks pretty good and is more easily interpreted - continue with this

<p><a name="missingness"></a></p>

### Investigating missingness

In [ ]:
# Get an overview of the feature set
all_data.describe()
# 37 numerical variables inc. SalePrice

In [ ]:
all_data.columns[all_data.isnull().any()]
# 25 features with missing data

In [ ]:
# View the columns with missing data
all_data.loc[:, ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType']]

In [ ]:
# Visualize missing data combinations
fig, ax = plt.subplots(figsize=(11.7, 8.27))
sns.heatmap(train_data.loc[:, ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType']].isnull(), cbar = False, ax=ax)

<p><a name="cleaning"></a></p>

### Data cleaning and preliminary feature selection
Most appropriate method of imputation has been selected for each feature based on missingness quantity, pattern, feature characteristics and feature importance

In [ ]:
# Replace the missing values from the basement data with random selections from the appropriate options
np.random.seed(0)
basement = all_data.loc[:,['BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',]]
basement[(basement.isnull().any(axis = 1)) & (basement.loc[:,'TotalBsmtSF'] != 0)] =\
basement[(basement.isnull().any(axis = 1)) & (basement.loc[:,'TotalBsmtSF'] != 0)].apply(lambda x: x.fillna(np.random.choice(x.dropna())), axis=0)

# Replace the NA's that correspond to no garage with "None"
basement.loc[basement.isnull().any(axis = 1),['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']] = "None"
all_data.loc[:,['BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',]] = basement

# Remove the singular missing completely at random observation in the electrical column
all_data = all_data.drop(all_data[all_data.loc[:,'Electrical'].isnull()].index[0], axis = 0)

# Replace all NA values that actually mean no fireplaces with 'None'
all_data.loc[(all_data['FireplaceQu'].isnull()) & (all_data['Fireplaces'] == 0),'FireplaceQu'] = "None"

# Replace all NA values that actually mean no garage with 'None'
all_data.loc[(all_data['GarageType'].isnull()) &(all_data['GarageArea'] == 0),'GarageType'] = 'None'
all_data.loc[(all_data['GarageFinish'].isnull()) &(all_data['GarageArea'] == 0),'GarageFinish'] = 'None'
all_data.loc[(all_data['GarageQual'].isnull()) &(all_data['GarageArea'] == 0),'GarageQual'] = 'None'

# Impute for the very few missing values in the GarageQual and GarageFinish
GarFin_index = (all_data[(all_data.GarageArea!=0) & (all_data.GarageFinish.isnull())]['GarageFinish']).index
GarQual_index  = (all_data[(all_data.GarageArea!=0) & (all_data.GarageQual.isnull())]['GarageQual']).index
all_data.loc[GarQual_index,'GarageQual'] = np.random.choice(['TA', 'Fa', 'Gd','Ex', 'Po'])
all_data.loc[GarFin_index,'GarageFinish'] = np.random.choice(['Fin','RFn','Unf'])

# Impute using the mean/median for the very few missing values in the other garage columns
all_data.loc[(all_data['GarageYrBlt'].isnull()) &(all_data['GarageArea'] == 0),'GarageYrBlt'] = 'None'
all_data.loc[2576,'GarageArea'] = all_data.GarageArea.mean()
all_data.loc[2576,'GarageCars'] = all_data.GarageCars.mean()
all_data.loc[[2126,2576],'GarageYrBlt'] = all_data.GarageYrBlt[all_data.GarageYrBlt != 'None'].median()

# Replace all NA values that actualy mean no pool with "None"
all_data.loc[(all_data['PoolQC'].isnull()) & (all_data['PoolArea'] == 0),'PoolQC'] = 'None'

# Impute for the very few missing values in the PoolQC column
poolna_index = (all_data[(all_data.PoolArea!=0) & (all_data.PoolQC.isnull())]['PoolQC']).index
all_data.loc[poolna_index[0],'PoolQC'] = np.random.choice(['Ex','Gd','Fa'])
all_data.loc[poolna_index[1],'PoolQC'] = np.random.choice(['Ex','Gd','Fa'])
all_data.loc[poolna_index[2],'PoolQC'] = np.random.choice(['Ex','Gd','Fa'])
all_data.loc[poolna_index,'PoolQC']

# Replace all NA values that actualy mean no fence with "None"
all_data['Fence'].fillna(value='None', inplace=True)

# Replace the only NA value with the majority value Typ
all_data['Functional'].fillna(value='Typ', inplace=True)

# Replace the remaining couple of NA values in the BsmtBath columns with values that preserve the column means
all_data.loc[2120, 'BsmtFullBath'] = 1
all_data.loc[2188, 'BsmtFullBath'] = 0
all_data['BsmtHalfBath'].fillna(value=0, inplace=True)

# Replace all NA values that actualy mean no fence with "None"
all_data.loc[(all_data.MiscFeature.isnull()) & (all_data.MiscVal == 0),'MiscFeature'] = 'None'

# Replace the missing value in MicsFeature according to its corresponding MiscValue
all_data.loc[all_data.MiscFeature.isnull(), 'MiscFeature'] = 'Gar2'

# Replace missing values in the Exterior columns with majority values
all_data.loc[all_data.Exterior1st.isnull(),'Exterior1st'] = 'VinylSd'
all_data.loc[all_data.Exterior2nd.isnull(),'Exterior2nd'] = 'VinylSd'
all_data.loc[all_data.MSZoning.isnull(),'MSZoning'] = 'RL'
all_data.loc[all_data.SaleType.isnull(),'SaleType'] = 'WD'
all_data.loc[all_data.KitchenQual.isna(), 'KitchenQual'] = 'Ta'

In [ ]:
# For those columns with a larger number of truly missing values, randomly impute
all_lf_index = (all_data[all_data.LotFrontage.isnull()]).index
for idx in all_lf_index:
    all_data.loc[idx,'LotFrontage'] = np.random.choice(all_data.LotFrontage.dropna(), replace = False)

all_mvt_index = (all_data[all_data.MasVnrType.isnull()]).index
for idx in all_mvt_index:
    all_data.loc[idx,'MasVnrType'] = np.random.choice(['None', 'BrkFace', 'Stone'], replace = True)

all_mva_index = (all_data[all_data.MasVnrArea.isnull()]).index
for idx in all_mva_index:
    all_data.loc[idx,'MasVnrArea'] = np.random.choice(all_data.MasVnrArea.dropna(), replace = False)

In [ ]:
# Save a copy of the dataframe at this point (before dropping any columns) 
# Get rid of those columns that take almost exlcusively one value, and hence will ofer little to no information
all_data_no_drop = all_data.copy()
all_data = all_data.drop(["Street","Alley","Utilities", "BldgType", 'HouseStyle', 'YearBuilt', 'RoofMatl', 'BsmtFinSF1', 'BsmtFinSF2', 'Heating', 'GarageCond'], axis=1)

In [ ]:
# Check that all NaN's have been removed
all_data.loc[:,all_data.isnull().any()].isnull().sum()

##### Feature Selection

In [ ]:
# Combine some columns to further reduce down feature space
all_data["OthrRmsAbvGrd"] = all_data.TotRmsAbvGrd - all_data.KitchenAbvGr - all_data.BedroomAbvGr
all_data['Total_Porch_SF'] = all_data['WoodDeckSF']+all_data['OpenPorchSF']+all_data['EnclosedPorch']+all_data['3SsnPorch']+all_data['ScreenPorch']
all_data['TotalSf'] = all_data['TotalBsmtSF'] + all_data['GrLivArea'] + all_data['LowQualFinSF']
all_data['TotalBath'] = all_data['FullBath'] + all_data['BsmtFullBath'] + (0.5*all_data['HalfBath']) + (0.5*all_data['BsmtHalfBath'])
all_data['Pool'] = all_data['PoolArea'].apply(lambda x: 1 if x>0 else 0)

# These are being dropped because they're combined into new columns or because of very little correlation
all_data = all_data.drop(['TotRmsAbvGrd', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', \
                          'WoodDeckSF', 'OpenPorchSF', '2ndFlrSF', '1stFlrSF', 'TotalBsmtSF',\
                          'Condition1','Condition2','FullBath','BsmtFullBath','HalfBath','BsmtHalfBath',\
                          'BsmtUnfSF','MiscVal','LowQualFinSF','PoolArea','PoolQC','MiscFeature','LandSlope',\
                          'GarageYrBlt','GrLivArea','GarageCars'], axis = 1)

<p><a name="dummifying"></a></p>

In [ ]:
# Dummify all categorical columns, drop one from each set of dummies and the original categorical column
pd.set_option("display.max_columns", 500)
all_categorical = all_data.select_dtypes(exclude=[np.number]).columns
# mergedCat = all_data[all_categorical]
# mergedCat = pd.get_dummies(mergedCat,drop_first=True)
# dummified_full = pd.concat([all_data,mergedCat],axis=1)
# dummified_full = dummified_full.drop(all_categorical,axis=1)
dummified_full = pd.read_csv("data/train_sam.csv", sep='\t')
dummified_full = dummified_full.drop('GarageYrBlt', axis = 1)
dummified_full = dummified_full.drop(all_categorical,axis=1)
dummified_full = dummified_full.drop('Unnamed: 0', axis = 1)

In [ ]:
# Investigate non-dummified feature correlation with the target
pd.set_option("display.max_rows", 500)
all_data[all_data.loc[:,'SalePrice'] != 0].corr().loc['SalePrice'].abs().sort_values(ascending=False)

In [ ]:
# Investigate dummified feature correlation with the target
pd.set_option("display.max_rows", 500)
dummified_full[dummified_full.loc[:,'SalePrice'] != 0].corr().loc['SalePrice'].abs().sort_values(ascending=False)

<p><a name="modelling"></a></p>

### Model fitting

<p><a name="diagnostics"></a></p>

#### Diagnostics, multicollinearity, feature selection

In [ ]:
# Define a function for returning adjusted R squared, as well as checking the assumptions of linear regression
from statsmodels.stats.stattools import durbin_watson
def diagnostics(model,x,y):
    # Making adjusted R squared
    yhat = model.predict(x)
    SS_Residual = sum((y-yhat)**2)
    SS_Total = sum((y-np.mean(y))**2)
    r_squared = 1-(float(SS_Residual)/float(SS_Total))
    adjusted_r_squared = 1 - ((1-r_squared)*(len(y)-1))/(len(y)-x.shape[1]-1)
    return(adjusted_r_squared)
    
    # Testing assumptions
    ## Homoscedasticity and independent errors
    fig2, ax2 = plt.subplots(figsize=(11.7, 8.27))
    sns.scatterplot(x = yhat, y = y-yhat, ax=ax2).set_title("Residuals vs actual values")
    plt.xlabel("Actual values of target")
    plt.ylabel("Value of residuals")
    print("Durbin-Watson test statistic: {}".format(sm.stats.stattools.durbin_watson(resids = y-yhat)))
    
    ## Normality of residuals
    fig1, ax1 = plt.subplots(figsize=(11.7, 8.27))
    stats.probplot(y-yhat, dist=stats.norm, plot=ax1)
    
    # Adjusted R squared    
    print("Adjusted R^2: {}".format(adjusted_r_squared))

In [ ]:
# Re-split the all_data frame into train and test sets
train = dummified_full[dummified_full.SalePrice > 0]
test = dummified_full[dummified_full.SalePrice == 0]
test = test.drop('SalePrice',axis=1)

X_train = train.drop('SalePrice',axis=1)
Y_train = np.log(train.SalePrice)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) 

In [ ]:
# Use feature ranking with recursive feature elimination and cross-validation from Sklearn
import sklearn
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn import linear_model

ols = linear_model.LinearRegression()
rfecv = RFECV(estimator=ols, step=1, cv=KFold(10),scoring='r2')
rfecv.fit(x_train, y_train)

house_features = rfecv.support_
dummified_full.iloc[:,~house_features] #feature selection from RFECV method, columns to possibly drop

In [ ]:
# Investigate multi-collinearity between non-dummified features
temp = all_data[all_data.select_dtypes(include=[np.number]).columns].drop('SalePrice',axis=1)

scores = {}
ols2 = linear_model.LinearRegression()
from sklearn.metrics import r2_score
for feature_name in temp.columns:
                dummified_full2     = temp.copy()
                feature = dummified_full2[feature_name].copy()
                dummified_full2.drop(feature_name, axis=1, inplace=True)
                ols2.fit(dummified_full2, feature)
                scores[feature_name] = ols2.score(dummified_full2, feature)   

In [ ]:
# Visualize multi-collinearity between non-dummified features
plt.figure(figsize=(15,7))
g = sns.barplot(x='index', y='R2', data=pd.DataFrame(scores, index=['R2']).T.reset_index())
plt.xticks(rotation=70)
plt.title('$R^2$ of a continuous feature against the other features')

<p><a name="linear models"></a></p>

#### Linear Models

In [ ]:
# Use a simple multiple linear regression as a baseline, even though multi-collinearity 
# and curse of dimensionlity (many features) indicates this may be a bad choice
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression,HuberRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

linR = LinearRegression()
linR.fit(x_train, y_train)
linR_scores = cross_val_score(linR, x_train, y_train, cv=5)

Results_linR = pd.DataFrame({'CV':np.mean(linR_scores), 
                        'Adjusted': diagnostics(linR,x_train,y_train),
                        'Train':linR.score(x_train, y_train), 
                        'Test':linR.score(x_test, y_test)}, index = ['MLR'])

In [ ]:
# Try a Huber Regression (with a grid search) - this should handle any outliers well 
hubR = HuberRegressor()
params = {'alpha':10**np.linspace(-2,2,10), 'epsilon': np.linspace(1,2,10)}
grid = GridSearchCV(estimator=hubR, param_grid=params, cv=5)
grid.fit(x_train, y_train)
print(grid.best_params_)

hubR_optimal = grid.best_estimator_
Results_hubr = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(hubR_optimal,x_train,y_train),
                       'Train':hubR_optimal.score(x_train, y_train),
                       'Test':hubR_optimal.score(x_test, y_test)}, index = ['Huber'])

In [ ]:
# Try a Lasso penalized regression - this has the advantage of helping with feature selection
# Find the optimum lambda
lr = Lasso(normalize=True)

params = {'alpha':10**np.linspace(-4,-1,100)}
grid = GridSearchCV(estimator=lr, param_grid=params, cv=5)

grid.fit(x_train, y_train)
print(grid.best_params_)
print(grid.best_score_)
lr_optimal = grid.best_estimator_

Results_lasso = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(lr_optimal,x_train,y_train),
                       'Train':lr_optimal.score(x_train, y_train),
                       'Test':lr_optimal.score(x_test, y_test)}, index = ['Lasso'])

In [ ]:
# Use bootstrapping to generate confidence intervals for the coefficient estimates from the
# above regression. Any that span zero could be targets for removal
coefs = {}
intercepts = {}

for i in range(1000):
    rIndex = np.random.choice(x_train.index, 400, replace=False) # randomly select 400 samples 1000 times   
    sampled_x = x_train.loc[rIndex]
    sampled_y = y_train.loc[rIndex]

    grid.best_estimator_.fit(sampled_x, sampled_y)
    coefs[i] = grid.best_estimator_.coef_
    intercepts[i] = grid.best_estimator_.intercept_
coefs = pd.DataFrame(coefs)
intercepts= pd.Series(intercepts)
coefs_avg = coefs.apply('mean', axis =1)
coefs_sd = coefs.apply('std',axis = 1)

In [ ]:
plt.figure(figsize=(15,7))
coefs_lowlim = coefs_avg - 2*coefs_sd
coefs_uplim = coefs_avg + 2*coefs_sd
coefs_conf = pd.DataFrame({'lower': coefs_lowlim,'upper':coefs_uplim, 'boundary': np.where((coefs_lowlim <0) & (coefs_uplim >0),0,1)})
features_index = coefs_conf[coefs_conf.lower >0].index
x_train.columns[features_index]
# coefs_conf.loc[:,['lower','upper']].plot.line() # Plot if required

In [ ]:
# # Plot the magnitudes of the coefficients from the optimum Lasso model that are non-zero
# plt.figure(figsize=(15,7))
# feature_coefs = list(zip(x_train.columns[lr_optimal.coef_!=0], abs(lr_optimal.coef_[lr_optimal.coef_!=0])))
# dtype = [('feature', 'S10'), ('coefs', 'float')]
# feature_coefs = np.array(feature_coefs, dtype=dtype)
# feature_sort = np.sort(feature_coefs, order='coefs')[::-1]
# name, score = zip(*list(feature_sort))
# opt_coefs = pd.DataFrame({'name':name,'score':score})
# plt.bar(x=opt_coefs.name, height=opt_coefs.score)
# plt.xticks(rotation=70)

In [ ]:
# Try a ridge regression
# Find the optimum lambda
rd = Ridge(normalize=True)

params = {'alpha':10**np.linspace(-2,2,100)}
grid = GridSearchCV(estimator=rd,param_grid=params,cv=5)

grid.fit(x_train,y_train)
print(grid.best_params_)
rd_optimal = grid.best_estimator_

Results_ridge = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(rd_optimal,x_train,y_train),
                       'Train':rd_optimal.score(x_train, y_train),
                       'Test':rd_optimal.score(x_test, y_test)}, index = ['Ridge'])

In [ ]:
# Try an elasticnet regression
# FInd the optimum lambda and rho
en = ElasticNet(max_iter=10000,normalize=True)

params = {'alpha':10**np.linspace(-4,0,20),'l1_ratio':np.linspace(0.00001,0.001,20)}
grid = GridSearchCV(estimator=en,param_grid=params,cv=5)

grid.fit(x_train,y_train)
print(grid.best_params_)
en_optimal = grid.best_estimator_

Results_en = pd.DataFrame({'CV': grid.best_score_,
                       'Adjusted': diagnostics(en_optimal,x_train,y_train),
                       'Train':en_optimal.score(x_train, y_train),
                       'Test':en_optimal.score(x_test, y_test)}, index = ['ElasticNet'])

In [ ]:
# Kaggle Submission #1
# Using the results of the above, get a baseline score from submitting the elasticnet 
# model, with no additional feature selection (other than that effected by the L1 penalty)
sub1 = pd.DataFrame(np.exp(grid.predict(test)),index=test.index+1,columns=['SalePrice'])
sub1.to_csv('sub1.csv',sep=',')

<p><a name="optimal features"></a></p>

#### Optimal feature selection and model re-fitting

In [ ]:
# Visualize the development of the model coefficients in Lasso regression
alphas = 10**np.linspace(-6,-2,100)
lr.set_params(normalize=True)
coefs  = []

for alpha in alphas:
        lr.set_params(alpha=alpha)
        lr.fit(x_train, y_train)  
        coefs.append(lr.coef_)
        
coefs = pd.DataFrame(coefs, index = alphas, columns = x_train.columns)

plt.figure(figsize=(15,7))
for name in coefs.columns:
    plt.plot(coefs.index, coefs[name], label=name)

plt.xlabel(r'hyperparameter $\lambda$')
plt.ylabel(r'slope values')

In [ ]:
# Find the ~10 coefficients that take the longest to go to zero
coefs[(coefs.index>0.001)][coefs[(coefs.index>0.001)]!=0].notnull().sum()

######  - The optimal_features set below is made using the results of the above cell. 
######  - Optimal_features_2 contains the coefficients that are non-zero at the optimum lambda found from the lasso grid search. 
###### - Optimal_features_3 contains from the coefficients of the optimum lasso model with confidence intervals that do not span zero 

In [ ]:
# Make new feature spaces using only those features that Lasso regression suggests have the
# most explanatory power
optimal_features = x_train.loc[:,['OverallQual','YearRemodAdd','GarageArea','Fireplaces','OthrRmsAbvGrd','TotalBath','MSZoning_RM','MSZoning_RL','MSZoning_RH','MSZoning_FV','CentralAir_Y','FireplaceQu_Gd','FireplaceQu_TA','TotalSf']]
optimal_features2 = x_train.loc[:,x_train.columns[lr_optimal.coef_!=0]]
optimal_features3 = x_train.loc[:,x_train.columns[features_index]]
optimal_features_test = test.loc[:,['OverallQual','YearRemodAdd','GarageArea','Fireplaces','OthrRmsAbvGrd','TotalBath','MSZoning_RM','MSZoning_RL','MSZoning_RH','MSZoning_FV','CentralAir_Y','FireplaceQu_Gd','FireplaceQu_TA','TotalSf']]
optimal_features_test2 = test.loc[:,x_train.columns[lr_optimal.coef_!=0]]
optimal_features_test3 = test.loc[:,x_train.columns[features_index]]

In [ ]:
# Run linear regression with a reduced feature set
linR = LinearRegression()
linR.fit(optimal_features,y_train)
linR_scores = cross_val_score(linR,optimal_features,y_train,cv=5)
Results_linR_opt = pd.DataFrame({'CV':np.mean(linR_scores), 
                        'Adjusted': diagnostics(linR, optimal_features,y_train),
                        'Train':linR.score(optimal_features, y_train), 
                        'Test':linR.score(x_test.loc[:,list(optimal_features.columns)], y_test)}, 
                                index = ['MLR_opt'])

In [ ]:
# Run linear regression with a second reduced feature set
linR = LinearRegression()
linR.fit(optimal_features2,y_train)
linR_scores = cross_val_score(linR,optimal_features2,y_train,cv=5)
Results_linR_opt2 = pd.DataFrame({'CV':np.mean(linR_scores), 
                        'Adjusted': diagnostics(linR, optimal_features2,y_train),
                        'Train':linR.score(optimal_features2, y_train), 
                        'Test':linR.score(x_test.loc[:,list(optimal_features2.columns)], y_test)}, 
                                 index = ['MLR_opt2'])

In [ ]:
# Run linear regression with a third reduced feature set
linR = LinearRegression()
linR.fit(optimal_features3,y_train)
linR_scores = cross_val_score(linR,optimal_features3,y_train,cv=5)
Results_linR_opt3 = pd.DataFrame({'CV':np.mean(linR_scores), 
                        'Adjusted': diagnostics(linR, optimal_features3,y_train),
                        'Train':linR.score(optimal_features3, y_train), 
                        'Test':linR.score(x_test.loc[:,list(optimal_features3.columns)], y_test)}, 
                                 index = ['MLR_opt3'])

In [ ]:
# Run Huber Regression with a reduced feature set
hubR = HuberRegressor()
params = {'alpha':10**np.linspace(-4,2,10), 'epsilon': np.linspace(1,5,50)}
grid = GridSearchCV(estimator=hubR, param_grid=params, cv=5)
grid.fit(optimal_features, y_train)
hubR_optimal = grid.best_estimator_

Results_hubR_opt = pd.DataFrame({'CV':grid.best_score_, 
                        'Adjusted': diagnostics(hubR_optimal, optimal_features, y_train),
                        'Train':hubR_optimal.score(optimal_features, y_train), 
                        'Test':hubR_optimal.score(x_test.loc[:,list(optimal_features.columns)], y_test)}, 
                                index = ['hubR_opt'])

In [ ]:
# Run Huber Regression with a second reduced feature set
hubR = HuberRegressor()
params = {'alpha':10**np.linspace(-4,2,10), 'epsilon': np.linspace(1,5,50)}
grid = GridSearchCV(estimator=hubR, param_grid=params, cv=5)
grid.fit(optimal_features2, y_train)
hubR_optimal = grid.best_estimator_

Results_hubR_opt2 = pd.DataFrame({'CV': grid.best_score_, 
                        'Adjusted': diagnostics(hubR_optimal, optimal_features2, y_train),
                        'Train':hubR_optimal.score(optimal_features2, y_train), 
                        'Test':hubR_optimal.score(x_test.loc[:,list(optimal_features2.columns)], y_test)}, 
                                index = ['hubR_opt_2'])

In [ ]:
# Run Huber Regression with a third reduced feature set
hubR = HuberRegressor()
params = {'alpha':10**np.linspace(-2,2,10), 'epsilon': np.linspace(1,5,50)}
grid = GridSearchCV(estimator=hubR, param_grid=params, cv=5)
grid.fit(optimal_features3, y_train)
hubR_optimal = grid.best_estimator_

Results_hubR_opt3 = pd.DataFrame({'CV':grid.best_score_, 
                        'Adjusted': diagnostics(hubR_optimal, optimal_features3, y_train),
                        'Train':hubR_optimal.score(optimal_features3, y_train), 
                        'Test':hubR_optimal.score(x_test.loc[:,list(optimal_features3.columns)], y_test)}, 
                                index = ['hubR_opt_3'])

In [ ]:
# Run Lasso regression with a reduced feature set
lr = Lasso(normalize=True)

params = {'alpha':10**np.linspace(-6,-3,100)}
grid = GridSearchCV(estimator=lr,param_grid=params,cv=5)

grid.fit(optimal_features,y_train)
lr_optimal = grid.best_estimator_

Results_lasso_opt = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(lr_optimal,optimal_features,y_train),
                       'Train':lr_optimal.score(optimal_features, y_train),
                       'Test':lr_optimal.score(x_test.loc[:,list(optimal_features.columns)], y_test)}, 
                                  index = ['Lasso_opt'])

In [ ]:
# Run Lasso regression with a second reduced feature set
lr = Lasso(normalize=True)

params = {'alpha':10**np.linspace(-6,-3,100)}
grid = GridSearchCV(estimator=lr,param_grid=params,cv=5)

grid.fit(optimal_features2,y_train)
lr_optimal = grid.best_estimator_

Results_lasso_opt2 = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(lr_optimal,optimal_features2,y_train),
                       'Train':lr_optimal.score(optimal_features2, y_train),
                       'Test':lr_optimal.score(x_test.loc[:,list(optimal_features2.columns)], y_test)}, 
                                  index = ['Lasso_opt_2'])

In [ ]:
# Run Lasso regression with a third reduced feature set
lr = Lasso(normalize=True)

params = {'alpha':10**np.linspace(-6,-3,100)}
grid = GridSearchCV(estimator=lr,param_grid=params,cv=5)

grid.fit(optimal_features3,y_train)
lr_optimal = grid.best_estimator_

Results_lasso_opt3 = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(lr_optimal,optimal_features3,y_train),
                       'Train':lr_optimal.score(optimal_features3, y_train),
                       'Test':lr_optimal.score(x_test.loc[:,list(optimal_features3.columns)], y_test)}, 
                                  index = ['Lasso_opt_3'])

In [ ]:
# Run Ridge regression with a reduced feature set
rd = Ridge(normalize=True)

params = {'alpha':10**np.linspace(-4,2,100)}
grid = GridSearchCV(estimator=rd,param_grid=params,cv=5)

grid.fit(optimal_features,y_train)
rd_optimal = grid.best_estimator_

Results_ridge_opt = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(rd_optimal,optimal_features,y_train),
                       'Train':rd_optimal.score(optimal_features, y_train),
                       'Test':rd_optimal.score(x_test.loc[:,list(optimal_features.columns)], y_test)}, 
                                  index = ['Ridge_opt'])

In [ ]:
# Run Ridge regression with a second reduced feature set
rd = Ridge(normalize=True)

params = {'alpha':10**np.linspace(-4,2,100)}
grid = GridSearchCV(estimator=rd,param_grid=params,cv=5)

grid.fit(optimal_features2,y_train)
rd_optimal = grid.best_estimator_

Results_ridge_opt2 = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(rd_optimal,optimal_features2,y_train),
                       'Train':rd_optimal.score(optimal_features2, y_train),
                       'Test':rd_optimal.score(x_test.loc[:,list(optimal_features2.columns)], y_test)}, 
                                  index = ['Ridge_opt_2'])

In [ ]:
# Run Ridge regression with a third reduced feature set
rd = Ridge(normalize=True)

params = {'alpha':10**np.linspace(-4,2,100)}
grid = GridSearchCV(estimator=rd,param_grid=params,cv=5)

grid.fit(optimal_features3,y_train)
rd_optimal = grid.best_estimator_

Results_ridge_opt3 = pd.DataFrame({'CV':grid.best_score_,
                       'Adjusted': diagnostics(rd_optimal,optimal_features3,y_train),
                       'Train':rd_optimal.score(optimal_features3, y_train),
                       'Test':rd_optimal.score(x_test.loc[:,list(optimal_features3.columns)], y_test)}, 
                                  index = ['Ridge_opt_3'])

In [ ]:
# Kaggle Submission #2
# Compare the previous submission to the result obtained from the Ridge regression with
# the optimal_features reduced feature set
sub2 = pd.DataFrame(np.exp(grid.predict(optimal_features_test)),index=optimal_features_test.index+1,columns=['SalePrice'])
sub2.to_csv('sub2.csv',sep=',')

In [ ]:
# Run elasticnet regression with a reduced feature set
en = ElasticNet(max_iter=10000,normalize=True)

params = {'alpha':10**np.linspace(-5,-3,20),'l1_ratio':np.linspace(0.001,1,20)}
grid = GridSearchCV(estimator=en,param_grid=params,cv=5)

grid.fit(optimal_features,y_train)
en_optimal = grid.best_estimator_

Results_en_opt = pd.DataFrame({'CV':grid.best_score_, 
                        'Adjusted': diagnostics(en_optimal,optimal_features,y_train),
                        'Train':en_optimal.score(optimal_features, y_train), 
                        'Test':en_optimal.score(x_test.loc[:,list(optimal_features.columns)], y_test)}, 
                               index = ['ElasticNet_opt'])

In [ ]:
# Run elasticnet regression with a second reduced feature set
en = ElasticNet(max_iter=10000,normalize=True)

params = {'alpha':10**np.linspace(-5,-3,20),'l1_ratio':np.linspace(0.001,1,20)}
grid = GridSearchCV(estimator=en,param_grid=params,cv=5)

grid.fit(optimal_features2,y_train)
en_optimal = grid.best_estimator_

Results_en_opt2 = pd.DataFrame({'CV':grid.best_score_, 
                        'Adjusted': diagnostics(en_optimal,optimal_features2,y_train),
                        'Train':en_optimal.score(optimal_features2, y_train), 
                        'Test':en_optimal.score(x_test.loc[:,list(optimal_features2.columns)], y_test)}, 
                               index = ['ElasticNet_opt2'])

In [ ]:
# Run elasticnet regression with a third reduced feature set
en = ElasticNet(max_iter=10000,normalize=True)

params = {'alpha':10**np.linspace(-5,-3,20),'l1_ratio':np.linspace(0.001,1,20)}
grid = GridSearchCV(estimator=en,param_grid=params,cv=5)

grid.fit(optimal_features3,y_train)
en_optimal = grid.best_estimator_

Results_en_opt3 = pd.DataFrame({'CV':grid.best_score_, 
                        'Adjusted': diagnostics(en_optimal,optimal_features3,y_train),
                        'Train':en_optimal.score(optimal_features3, y_train), 
                        'Test':en_optimal.score(x_test.loc[:,list(optimal_features3.columns)], y_test)}, 
                               index = ['ElasticNet_opt3'])

<p><a name="linear model results"></a></p>

#### Linear model results

In [ ]:
linear_model_results = pd.concat([Results_linR, Results_hubr, Results_lasso, Results_ridge, Results_en,
                                  Results_linR_opt, Results_linR_opt2, Results_linR_opt3,
                                  Results_hubR_opt, Results_hubR_opt2,
                                  Results_lasso_opt, Results_lasso_opt2, Results_lasso_opt3,
                                  Results_ridge_opt, Results_ridge_opt2, Results_ridge_opt3, 
                                  Results_en_opt, Results_en_opt2, Results_en_opt3], axis = 0)

In [ ]:
linear_model_results

In [ ]:
#Kaggle Submission #4
# COmpare the previous elasticnet submission to this one, using a different reduced feature set
sub4 = pd.DataFrame(np.exp(grid.predict(optimal_features_test2)),index=optimal_features_test2.index+1,columns=['SalePrice'])
sub4.to_csv('sub4.csv',sep=',')

In [ ]:
#Kaggle Submission #5
# Using the bootstrapped values on the lasso regression
sub5 = pd.DataFrame(np.exp(grid.predict(optimal_features_test3)),index=optimal_features_test3.index+1,columns=['SalePrice'])
sub5.to_csv('sub5.csv',sep=',')

<p><a name="non-linear models"></a></p>

#### Non-linear models

In [ ]:
# Decision Tree
from sklearn import tree
tree_model = tree.DecisionTreeRegressor()

grid_para_tree = [{
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=60, num=15, dtype=int)
}]
tree_model.set_params(random_state=108)
grid_search_tree = GridSearchCV(tree_model, grid_para_tree, cv=10, scoring='r2', n_jobs=-1)
%time grid_search_tree.fit(x_train, y_train)

best_tree_model = grid_search_tree.best_estimator_
print(grid_search_tree.best_params_)

Results_decisiontree = pd.DataFrame({'CV':grid_search_tree.best_score_,
                       'Adjusted': diagnostics(best_tree_model,x_train,y_train),
                       'Train':best_tree_model.score(x_train, y_train),
                       'Test':best_tree_model.score(x_test, y_test)}, 
                                    index = ['Decision_tree'])
Results_decisiontree

In [ ]:
# Kaggle submission 3
# Compare how the tree model does, relative to all the linear models previously
sub4 = pd.DataFrame(np.exp(grid_search_tree.predict(test)),index=test.index+1,columns=['SalePrice'])
sub4.to_csv('sub4.csv',sep=',')

In [ ]:
# Run a Random Forest Model
from sklearn import ensemble

randomForest = ensemble.RandomForestRegressor()

grid_para_forest = [{'n_estimators': [25,50,100],'min_samples_leaf':range(1,10),'min_samples_split':np.linspace(2,30,15,dtype=int),'random_state':[42]}]

grid_search_forest = GridSearchCV(randomForest,grid_para_forest,scoring='r2',cv=5,n_jobs=-1)
%time grid_search_forest.fit(x_train,y_train)

best_forest_model = grid_search_forest.best_estimator_
print(grid_search_forest.best_params_)

Results_forest = pd.DataFrame({'CV':grid_search_forest.best_score_,
                       'Adjusted': diagnostics(best_forest_model,x_train,y_train),
                       'Train':best_forest_model.score(x_train, y_train),
                       'Test':best_forest_model.score(x_test, y_test)}, 
                              index = ['Random_forest'])

We have implemented some feature selection, but a strength of tree/forest models is their ability to identify those features that contribute most to predicting a given target. Hence make a second input data set, without dropping columns

In [ ]:
# Dummify the data set with no columns dropped
pd.set_option("display.max_columns", 500)
all_categorical_rf = all_data_no_drop.select_dtypes(exclude=[np.number]).columns
mergedCat_rf = all_data_no_drop[all_categorical_rf]
mergedCat_rf = pd.get_dummies(mergedCat_rf,drop_first=True)
dummified_full_rf = pd.concat([all_data_no_drop,mergedCat_rf],axis=1)
dummified_full_rf = dummified_full_rf.drop(all_categorical_rf,axis=1)

In [ ]:
# Split the data with no columns dropped into train and test
train_rf = dummified_full_rf[dummified_full_rf.SalePrice > 0]
test_rf = dummified_full_rf[dummified_full_rf.SalePrice == 0]
test = test_rf.drop('SalePrice',axis=1)

X_train_rf = train_rf.drop('SalePrice',axis=1)
Y_train_rf = np.log(train_rf.SalePrice)

# This section is useful to create dataframe comparing our train scores to the test scores
from sklearn.model_selection import train_test_split
x_train_rf, x_test_rf, y_train_rf, y_test_rf = train_test_split(X_train_rf, Y_train_rf, test_size=0.2, random_state=42

In [ ]:
# Run a Random Forest Model on the data with no columns dropped
from sklearn import ensemble

randomForest = ensemble.RandomForestRegressor()

grid_para_forest = [{'n_estimators': [25,50,100],'min_samples_leaf':range(1,10),'min_samples_split':np.linspace(2,30,15,dtype=int),'random_state':[42]}]

grid_search_forest = GridSearchCV(randomForest,grid_para_forest,scoring='r2',cv=5,n_jobs=-1)
%time grid_search_forest.fit(x_train_rf,y_train_rf)

best_forest_model = grid_search_forest.best_estimator_
print(grid_search_forest.best_params_)

Results_forest_no_drop = pd.DataFrame({'CV':grid_search_forest.best_score_,
                       'Adjusted': diagnostics(best_forest_model,x_train_rf,y_train_rf),
                       'Train':best_forest_model.score(x_train_rf, y_train_rf),
                       'Test':best_forest_model.score(x_test_rf, y_test_rf)}, 
                              index = ['Random_forest_no_drop'])

In [ ]:
# Extract feature importances from the random forest
feature_importance = list(zip(x_train.columns, best_forest_model.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
name, score = zip(*list(feature_sort))
pd.DataFrame({'name':name,'score':score}).plot.bar(x='name', y='score')
feature_sort

In [ ]:
# Submit the predictions of the random forest on the slightly reduced data set
sub3 = pd.DataFrame(np.exp(grid_search_forest.predict(test)),index=test.index+1,columns=['SalePrice'])
sub3.to_csv('sub3.csv',sep=',')

In [ ]:
# Submit the predictions of the random forest on the complete data set
sub6 = pd.DataFrame(np.exp(grid_search_forest.predict(test)),index=test.index+1,columns=['SalePrice'])
sub6.to_csv('sub6.csv',sep=',')

#### Gradient Boosting

In [ ]:
# Gradient boosting regressor using linear
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(verbose=1)

grid_para_boost_lr = [{
#     "learning_rate": np.linspace(start=0.01, stop=0.1, num=10),
    "n_estimators": [100, 500],
    "min_samples_leaf": range(1, 5),
    "min_samples_split": np.linspace(start=2, stop=10, num=5, dtype=int),
    "random_state": [42]}]
grid_search_boost = GridSearchCV(gbm, grid_para_boost_lr, scoring='r2', cv=5, n_jobs=3)
%time grid_search_boost.fit(x_train, y_train)

best_boost_model = grid_search_boost.best_estimator_
print(grid_search_boost.best_params_)

Results_boost = pd.DataFrame({'CV':grid_search_boost.best_score_,
                       'Adjusted': diagnostics(best_boost_model, x_train, y_train),
                       'Train':best_boost_model.score(x_train, y_train),
                       'Test':best_boost_model.score(x_test, y_test)}, 
                              index = ['Gradient_boosting'])

In [ ]:
# Gradient boosting regressor using huber
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(loss = 'huber', verbose = 1, subsample = 0.9, learning_rate = 0.2)

grid_para_boost_hu = [{
    "alpha": np.linspace(start = 0.75, stop = 0.95, num = 10),
#     "learning_rate": np.linspace(start=0.01, stop=0.1, num=10),
    "n_estimators": [100, 300],
    "min_samples_leaf": range(1, 5),
    "min_samples_split": np.linspace(start=2, stop=10, num=5, dtype=int),
    "random_state": [42]}]
grid_search_boost = GridSearchCV(gbm, grid_para_boost_hu, scoring='r2', cv=5, n_jobs=3)
%time grid_search_boost.fit(x_train, y_train)

best_boost_model = grid_search_boost.best_estimator_
print(grid_search_boost.best_params_)

Results_boost_hubr = pd.DataFrame({'CV':grid_search_boost.best_score_,
                       'Adjusted': diagnostics(best_boost_model, x_train, y_train),
                       'Train':best_boost_model.score(x_train, y_train),
                       'Test':best_boost_model.score(x_test, y_test)}, 
                              index = ['Gradient_boosting_hubr'])

In [ ]:
from sklearn.svm import SVR
svr = SVR()
paramDict = [{'C':np.linspace(1,100,20),'gamma':np.linspace(1e-6,1e-2,10)}]

grid_search_svr = GridSearchCV(svr, paramDict, cv=5, return_train_score = True, n_jobs=-1)
%time grid_search_svr.fit(x_train,y_train)

best_svr_model = grid_search_svr.best_estimator_
print(grid_search_svr.best_params_)

Results_svr = pd.DataFrame({'CV':grid_search_svr.best_score_,
                       'Adjusted': diagnostics(best_svr_model, x_train, y_train),
                       'Train':best_svr_model.score(x_train, y_train),
                       'Test':best_svr_model.score(x_test, y_test)}, 
                              index = ['Svr'])

In [ ]:
# Submit the predictions of the gradient boost regression
sub9 = pd.DataFrame(np.exp(grid_search_boost.predict(test)),index=test.index+1,columns=['SalePrice'])
sub9.to_csv('sub9.csv',sep=',')

<p><a name="all results"></a></p>

#### All Results

In [ ]:
all_results = pd.concat([linear_model_results, Results_decisiontree, Results_forest, 
                         Results_forest_no_drop,
                         Results_boost, Results_boost_hubr, Results_svr], axis = 0)

In [ ]:
all_results

In [ ]:
all_results.to_csv("All_Results.csv")